#### This jupyter notebook contains code to get detailed information of businesses in Yelp Buisness Details API

In [2]:
# Importing packages
import requests
from pandas import json_normalize
import pandas as pd
import requests
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib as plt

## Part 1: Get Business ID

#### The business details API needs the business's id as input, so I need to first use the normal Yelp Buisness Search API to get id. The following part of code is the same as the Yelp_API_Business_Search_codes.ipynb

I use a config file to store my api key. For people aiming to run this notebook, you can apply you own Yelp API key in the Yelp developer portal and store it in a config file in the same folder.

In [3]:
# Extracting Yelp Key
from configparser import ConfigParser
config = ConfigParser()
config.read('config.ini')

API_KEY = config["YELP"]["api_key"]

In [5]:
# The url accessing to the Yelp Business Details API
endpoint_url_start = "https://api.yelp.com/v3/businesses/search"

In [6]:
# Initial the dataframe to store business details info
galveston_locations = pd.DataFrame(columns=['Name','Address','Coordinates',"id"])
hilo_locations = pd.DataFrame(columns=['Name','Address','Coordinates',"id"])

In [7]:
# Get the number of businesses in Galveston
res = requests.get(url=endpoint_url_start,
                        params = {"location":"Galveston, TX"},
                        headers={"Authorization":API_KEY}
                        )
response = json.loads(res.content)
total_galveston = response["total"]

In [8]:
# Get the number of businesses in Hilo
res = requests.get(url=endpoint_url_start,
                        params = {"location":"Hilo, HI"},
                        headers={"Authorization":API_KEY}
                        )
response = json.loads(res.content)
total_hilo = response["total"]
total_hilo

378

In [9]:
# function to append the business detailed info to the dataframe
# Takes care of null returned from API request
def append_place(response, df):
    l = len(df)
    for i, val in enumerate(response["businesses"]):
        df.loc[l + i] = [val.get('name', np.nan), 
                         val.get('location', np.nan).get("display_address"), 
                         val.get('coordinates', np.nan),
                         val.get("id",np.nan)]

In [10]:
# Use the function I defined above, I extract the info from the Yelp API and store them into the dataframe I made.
# total_galveston and total_hilo are the total number of businesses in each city.
location = {"Galveston, TX": [galveston_locations, total_galveston], "Hilo, HI": [hilo_locations, total_hilo]}
# Go through Galveston and Hilo
for i,v in location.items():
    # Here, I got 20 business info each time until I got all business in a city. 
    # The reason is that one page of Yelp API contains 20 businesses so I cannot get more than 20 business details each time. 
    for t in range(0, v[1], 20):
        res = requests.get(url=endpoint_url_start,
                        params = {"location":i, "offset":t},
                        headers={"Authorization":API_KEY}
                        )
        response = json.loads(res.content)
        append_place(response, v[0])

In [11]:
# Originally, the address is not in string form. I use the following code to turn the address to string form
full_address_galveston = []
for i in galveston_locations["Address"]:
    full_address_galveston.append(" ".join([j for j in i]))
galveston_locations["Address"] = full_address_galveston

full_address_hilo = []
for i in hilo_locations["Address"]:
    full_address_hilo.append(" ".join([j for j in i]))
hilo_locations["Address"] = full_address_hilo

In [12]:
#Extract Galveston's latitude and longitude from the coordinate column as columns
gal_latitude = []
gal_longitude = []
for i in galveston_locations["Coordinates"]:
    gal_latitude.append(i["latitude"])
    gal_longitude.append(i["longitude"])
galveston_locations["latitude"] = gal_latitude
galveston_locations["longitude"] = gal_longitude

In [13]:
#Extract Hilo's latitude and longitude from the coordinate column as columns
hilo_latitude = []
hilo_longitude = []
for i in hilo_locations["Coordinates"]:
    hilo_latitude.append(i["latitude"])
    hilo_longitude.append(i["longitude"])
hilo_locations["latitude"] = hilo_latitude
hilo_locations["longitude"] = hilo_longitude

## Part 2: Get Galveston's Business Details

In [21]:
# Use a list to store all json files of Galveston getting from Business details API
# for debug purpose, not using the data here
galveston_json_list = []
endpoint_url = "https://api.yelp.com/v3/businesses/"
for i in galveston_locations["id"]:
    endpoint_url_detail = endpoint_url + i
    res = requests.get(url=endpoint_url_detail,
                        headers={"Authorization":API_KEY}
                        )
    response = json.loads(res.content)
    galveston_json_list.append(response)

In [46]:
# Use a Pandas Dataframe to store all json files of Galveston getting from Business details API
galveston_df_list = []
endpoint_url = "https://api.yelp.com/v3/businesses/"
# Use ID as key to extract data
for i in galveston_locations["id"]:
    endpoint_url_detail = endpoint_url + i
    res = requests.get(url=endpoint_url_detail,
                        headers={"Authorization":API_KEY}
                        )
    df = json_normalize(res.json())
    galveston_df_list.append(df)
galveston_details = pd.concat(galveston_df_list)
galveston_details

,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,...,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude,messaging.url,messaging.use_case_text,special_hours,error.code,error.description
0,YAASmrPYP5Hp31kqadx__w,cajun-greek-galveston,Cajun Greek,https://s3-media3.fl.yelpcdn.com/bphoto/LKcJci...,True,False,https://www.yelp.com/biz/cajun-greek-galveston...,+14097447041,(409) 744-7041,999.0,...,TX,"[2226 61st St, Galveston, TX 77551]",,29.276939,-94.831641,NaN,NaN,NaN,NaN,NaN
0,CyhcrbaQ4Wjbud9rd9HJnQ,shrimp-n-stuff-galveston-4,Shrimp N Stuff,https://s3-media4.fl.yelpcdn.com/bphoto/oFhJ9E...,True,False,https://www.yelp.com/biz/shrimp-n-stuff-galves...,+14097632805,(409) 763-2805,1441.0,...,TX,"[3901 Ave O, Galveston, TX 77550]",,29.288740,-94.808440,NaN,NaN,NaN,NaN,NaN
0,doSYFMDc3somcL4YKQb2FA,gypsy-joynt-galveston-2,Gypsy Joynt,https://s3-media1.fl.yelpcdn.com/bphoto/0fZzqv...,True,False,https://www.yelp.com/biz/gypsy-joynt-galveston...,+14094972069,(409) 497-2069,622.0,...,TX,"[6105 Stewart Rd, Galveston, TX 77551]",,29.274172,-94.831648,NaN,NaN,NaN,NaN,NaN
0,C4dQLeweVAez_gVuk9A60g,seahorse-grill-galveston-2,Seahorse Grill,https://s3-media2.fl.yelpcdn.com/bphoto/ZWXpfT...,True,False,https://www.yelp.com/biz/seahorse-grill-galves...,+14097418400,(409) 741-8400,237.0,...,TX,"[3802 Cove View Blvd, Ste C, Galveston, TX 77554]",,29.244487,-94.868038,NaN,NaN,NaN,NaN,NaN
0,oxKepiR0j31RPOL9KcNwdQ,bronco-burritos-galveston,Bronco Burritos,https://s3-media2.fl.yelpcdn.com/bphoto/ouEDxP...,True,False,https://www.yelp.com/biz/bronco-burritos-galve...,+14097629344,(409) 762-9344,289.0,...,TX,"[4917 Broadway St, Galveston, TX 77551]",,29.291766,-94.822378,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,_XheZCn-hfosWUEqIFG7tA,the-shiver-shack-league-city,The Shiver Shack,https://s3-media1.fl.yelpcdn.com/bphoto/qLIka0...,True,False,https://www.yelp.com/biz/the-shiver-shack-leag...,+18327542330,(832) 754-2330,6.0,...,TX,"[League City, TX 77573]",,29.501650,-95.056980,NaN,NaN,NaN,NaN,NaN
0,MvEBOqqAgMEd4AZ2KR8tww,the-wet-whistle-galveston,The Wet Whistle,https://s3-media3.fl.yelpcdn.com/bphoto/-Cl6AL...,True,False,https://www.yelp.com/biz/the-wet-whistle-galve...,+14095264433,(409) 526-4433,1.0,...,TX,"[21502 San Luis Pass, Galveston, TX 77554]",,29.141583,-95.040493,https://www.yelp.com/raq/MvEBOqqAgMEd4AZ2KR8tw...,Message the Business,NaN,NaN,NaN
0,TYsi4SYrkmLiGtPP5LE10g,bar-43-galveston,Bar 43,https://s3-media4.fl.yelpcdn.com/bphoto/8JNNli...,True,False,https://www.yelp.com/biz/bar-43-galveston?adju...,+14096322105,(409) 632-2105,1.0,...,TX,"[4302 Ave S, Galveston, TX 77550]",,29.280968,-94.810836,https://www.yelp.com/raq/TYsi4SYrkmLiGtPP5LE10...,Message the Business,NaN,NaN,NaN
0,SsmOfFeY0wdPB7ED26JTuQ,the-kings-court-freeport,The King's Court,https://s3-media3.fl.yelpcdn.com/bphoto/divNw7...,True,False,https://www.yelp.com/biz/the-kings-court-freep...,+17135182098,(713) 518-2098,3.0,...,TX,"[528 Gulf Blvd, Freeport, TX 77541]",,28.966447,-95.364985,NaN,NaN,NaN,NaN,NaN


In [23]:
# Define a function to help as detect NaN value
def isNaN(num):
    return num != num

In [130]:
# Get all rows that did not get accurate information due to "too many requests" error
# I need to replace these rows with error messages to rows with accurate business details information
galveston_nan = []
for i in range(len(list(galveston_details["name"]))):
    if isNaN(list(galveston_details["name"])[i]):
        galveston_nan.append(i)
galveston_nan

[114]

In [136]:
# Get the accurate information of the rows with problems that I located above
endpoint_url = "https://api.yelp.com/v3/businesses/"
# I use the number "114" here because my previous function returns 114 as the index number of
# the row with problems. The rows that have problems may not be the same each time, so it is
# necessary to change 114 to other numbers based on the results of the previous cell
id_num = galveston_locations.iloc[114]["id"]
endpoint_url_detail = endpoint_url + id_num
res = requests.get(url=endpoint_url_detail,
                        headers={"Authorization":API_KEY}
                        )
galveston_nan_detail = json_normalize(res.json())
galveston_nan_detail

,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,...,location.city,location.zip_code,location.country,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude,messaging.url,messaging.use_case_text
0,ob63TXvCteSjHYeVIx7fuA,el-jefe-mexican-food-galveston,El Jefe Mexican Food,https://s3-media2.fl.yelpcdn.com/bphoto/bRkH-O...,True,False,https://www.yelp.com/biz/el-jefe-mexican-food-...,+18326389168,(832) 638-9168,4,...,Galveston,77550,US,TX,"[3801 Broadway Ave J, Unit -B, Galveston, TX 7...",,29.295178,-94.809361,https://www.yelp.com/raq/ob63TXvCteSjHYeVIx7fu...,Message the Business


In [145]:
# Reset index and drop the blank line
galveston_details = galveston_details.reset_index()
galveston_details = galveston_details.drop(114)
galveston_details 

,index,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,...,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude,messaging.url,messaging.use_case_text,special_hours,error.code,error.description
0,0,YAASmrPYP5Hp31kqadx__w,cajun-greek-galveston,Cajun Greek,https://s3-media3.fl.yelpcdn.com/bphoto/LKcJci...,True,False,https://www.yelp.com/biz/cajun-greek-galveston...,+14097447041,(409) 744-7041,...,TX,"[2226 61st St, Galveston, TX 77551]",,29.276939,-94.831641,NaN,NaN,NaN,NaN,NaN
1,0,CyhcrbaQ4Wjbud9rd9HJnQ,shrimp-n-stuff-galveston-4,Shrimp N Stuff,https://s3-media4.fl.yelpcdn.com/bphoto/oFhJ9E...,True,False,https://www.yelp.com/biz/shrimp-n-stuff-galves...,+14097632805,(409) 763-2805,...,TX,"[3901 Ave O, Galveston, TX 77550]",,29.288740,-94.808440,NaN,NaN,NaN,NaN,NaN
2,0,doSYFMDc3somcL4YKQb2FA,gypsy-joynt-galveston-2,Gypsy Joynt,https://s3-media1.fl.yelpcdn.com/bphoto/0fZzqv...,True,False,https://www.yelp.com/biz/gypsy-joynt-galveston...,+14094972069,(409) 497-2069,...,TX,"[6105 Stewart Rd, Galveston, TX 77551]",,29.274172,-94.831648,NaN,NaN,NaN,NaN,NaN
3,0,C4dQLeweVAez_gVuk9A60g,seahorse-grill-galveston-2,Seahorse Grill,https://s3-media2.fl.yelpcdn.com/bphoto/ZWXpfT...,True,False,https://www.yelp.com/biz/seahorse-grill-galves...,+14097418400,(409) 741-8400,...,TX,"[3802 Cove View Blvd, Ste C, Galveston, TX 77554]",,29.244487,-94.868038,NaN,NaN,NaN,NaN,NaN
4,0,oxKepiR0j31RPOL9KcNwdQ,bronco-burritos-galveston,Bronco Burritos,https://s3-media2.fl.yelpcdn.com/bphoto/ouEDxP...,True,False,https://www.yelp.com/biz/bronco-burritos-galve...,+14097629344,(409) 762-9344,...,TX,"[4917 Broadway St, Galveston, TX 77551]",,29.291766,-94.822378,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,0,_XheZCn-hfosWUEqIFG7tA,the-shiver-shack-league-city,The Shiver Shack,https://s3-media1.fl.yelpcdn.com/bphoto/qLIka0...,True,False,https://www.yelp.com/biz/the-shiver-shack-leag...,+18327542330,(832) 754-2330,...,TX,"[League City, TX 77573]",,29.501650,-95.056980,NaN,NaN,NaN,NaN,NaN
167,0,MvEBOqqAgMEd4AZ2KR8tww,the-wet-whistle-galveston,The Wet Whistle,https://s3-media3.fl.yelpcdn.com/bphoto/-Cl6AL...,True,False,https://www.yelp.com/biz/the-wet-whistle-galve...,+14095264433,(409) 526-4433,...,TX,"[21502 San Luis Pass, Galveston, TX 77554]",,29.141583,-95.040493,https://www.yelp.com/raq/MvEBOqqAgMEd4AZ2KR8tw...,Message the Business,NaN,NaN,NaN
168,0,TYsi4SYrkmLiGtPP5LE10g,bar-43-galveston,Bar 43,https://s3-media4.fl.yelpcdn.com/bphoto/8JNNli...,True,False,https://www.yelp.com/biz/bar-43-galveston?adju...,+14096322105,(409) 632-2105,...,TX,"[4302 Ave S, Galveston, TX 77550]",,29.280968,-94.810836,https://www.yelp.com/raq/TYsi4SYrkmLiGtPP5LE10...,Message the Business,NaN,NaN,NaN
169,0,SsmOfFeY0wdPB7ED26JTuQ,the-kings-court-freeport,The King's Court,https://s3-media3.fl.yelpcdn.com/bphoto/divNw7...,True,False,https://www.yelp.com/biz/the-kings-court-freep...,+17135182098,(713) 518-2098,...,TX,"[528 Gulf Blvd, Freeport, TX 77541]",,28.966447,-95.364985,NaN,NaN,NaN,NaN,NaN


In [146]:
# Merge the row with the accurate info I missed before to the original dataset
galveston_details = galveston_details.iloc[:,1:]
galveston_details_final = pd.concat([galveston_details , galveston_nan_detail])
galveston_details_final = galveston_details_final.reset_index()
galveston_details_final

,index,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,...,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude,messaging.url,messaging.use_case_text,special_hours,error.code,error.description
0,0,YAASmrPYP5Hp31kqadx__w,cajun-greek-galveston,Cajun Greek,https://s3-media3.fl.yelpcdn.com/bphoto/LKcJci...,True,False,https://www.yelp.com/biz/cajun-greek-galveston...,+14097447041,(409) 744-7041,...,TX,"[2226 61st St, Galveston, TX 77551]",,29.276939,-94.831641,NaN,NaN,NaN,NaN,NaN
1,1,CyhcrbaQ4Wjbud9rd9HJnQ,shrimp-n-stuff-galveston-4,Shrimp N Stuff,https://s3-media4.fl.yelpcdn.com/bphoto/oFhJ9E...,True,False,https://www.yelp.com/biz/shrimp-n-stuff-galves...,+14097632805,(409) 763-2805,...,TX,"[3901 Ave O, Galveston, TX 77550]",,29.288740,-94.808440,NaN,NaN,NaN,NaN,NaN
2,2,doSYFMDc3somcL4YKQb2FA,gypsy-joynt-galveston-2,Gypsy Joynt,https://s3-media1.fl.yelpcdn.com/bphoto/0fZzqv...,True,False,https://www.yelp.com/biz/gypsy-joynt-galveston...,+14094972069,(409) 497-2069,...,TX,"[6105 Stewart Rd, Galveston, TX 77551]",,29.274172,-94.831648,NaN,NaN,NaN,NaN,NaN
3,3,C4dQLeweVAez_gVuk9A60g,seahorse-grill-galveston-2,Seahorse Grill,https://s3-media2.fl.yelpcdn.com/bphoto/ZWXpfT...,True,False,https://www.yelp.com/biz/seahorse-grill-galves...,+14097418400,(409) 741-8400,...,TX,"[3802 Cove View Blvd, Ste C, Galveston, TX 77554]",,29.244487,-94.868038,NaN,NaN,NaN,NaN,NaN
4,4,oxKepiR0j31RPOL9KcNwdQ,bronco-burritos-galveston,Bronco Burritos,https://s3-media2.fl.yelpcdn.com/bphoto/ouEDxP...,True,False,https://www.yelp.com/biz/bronco-burritos-galve...,+14097629344,(409) 762-9344,...,TX,"[4917 Broadway St, Galveston, TX 77551]",,29.291766,-94.822378,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,167,MvEBOqqAgMEd4AZ2KR8tww,the-wet-whistle-galveston,The Wet Whistle,https://s3-media3.fl.yelpcdn.com/bphoto/-Cl6AL...,True,False,https://www.yelp.com/biz/the-wet-whistle-galve...,+14095264433,(409) 526-4433,...,TX,"[21502 San Luis Pass, Galveston, TX 77554]",,29.141583,-95.040493,https://www.yelp.com/raq/MvEBOqqAgMEd4AZ2KR8tw...,Message the Business,NaN,NaN,NaN
167,168,TYsi4SYrkmLiGtPP5LE10g,bar-43-galveston,Bar 43,https://s3-media4.fl.yelpcdn.com/bphoto/8JNNli...,True,False,https://www.yelp.com/biz/bar-43-galveston?adju...,+14096322105,(409) 632-2105,...,TX,"[4302 Ave S, Galveston, TX 77550]",,29.280968,-94.810836,https://www.yelp.com/raq/TYsi4SYrkmLiGtPP5LE10...,Message the Business,NaN,NaN,NaN
168,169,SsmOfFeY0wdPB7ED26JTuQ,the-kings-court-freeport,The King's Court,https://s3-media3.fl.yelpcdn.com/bphoto/divNw7...,True,False,https://www.yelp.com/biz/the-kings-court-freep...,+17135182098,(713) 518-2098,...,TX,"[528 Gulf Blvd, Freeport, TX 77541]",,28.966447,-95.364985,NaN,NaN,NaN,NaN,NaN
169,170,ezMsWPV1G82ItSEZDFL_PA,lions-bar-b-q-katy,Lion's Bar-B-Q,https://s3-media4.fl.yelpcdn.com/bphoto/GlGFwt...,True,False,https://www.yelp.com/biz/lions-bar-b-q-katy?ad...,+18328200720,(832) 820-0720,...,TX,"[18309 Fm 529, Katy, TX 77449]",,29.875160,-95.770610,NaN,NaN,NaN,NaN,NaN


In [147]:
# Clean unused column
galveston_details_final = galveston_details_final.iloc[:,1:]
galveston_details_final

,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,...,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude,messaging.url,messaging.use_case_text,special_hours,error.code,error.description
0,YAASmrPYP5Hp31kqadx__w,cajun-greek-galveston,Cajun Greek,https://s3-media3.fl.yelpcdn.com/bphoto/LKcJci...,True,False,https://www.yelp.com/biz/cajun-greek-galveston...,+14097447041,(409) 744-7041,999.0,...,TX,"[2226 61st St, Galveston, TX 77551]",,29.276939,-94.831641,NaN,NaN,NaN,NaN,NaN
1,CyhcrbaQ4Wjbud9rd9HJnQ,shrimp-n-stuff-galveston-4,Shrimp N Stuff,https://s3-media4.fl.yelpcdn.com/bphoto/oFhJ9E...,True,False,https://www.yelp.com/biz/shrimp-n-stuff-galves...,+14097632805,(409) 763-2805,1441.0,...,TX,"[3901 Ave O, Galveston, TX 77550]",,29.288740,-94.808440,NaN,NaN,NaN,NaN,NaN
2,doSYFMDc3somcL4YKQb2FA,gypsy-joynt-galveston-2,Gypsy Joynt,https://s3-media1.fl.yelpcdn.com/bphoto/0fZzqv...,True,False,https://www.yelp.com/biz/gypsy-joynt-galveston...,+14094972069,(409) 497-2069,622.0,...,TX,"[6105 Stewart Rd, Galveston, TX 77551]",,29.274172,-94.831648,NaN,NaN,NaN,NaN,NaN
3,C4dQLeweVAez_gVuk9A60g,seahorse-grill-galveston-2,Seahorse Grill,https://s3-media2.fl.yelpcdn.com/bphoto/ZWXpfT...,True,False,https://www.yelp.com/biz/seahorse-grill-galves...,+14097418400,(409) 741-8400,237.0,...,TX,"[3802 Cove View Blvd, Ste C, Galveston, TX 77554]",,29.244487,-94.868038,NaN,NaN,NaN,NaN,NaN
4,oxKepiR0j31RPOL9KcNwdQ,bronco-burritos-galveston,Bronco Burritos,https://s3-media2.fl.yelpcdn.com/bphoto/ouEDxP...,True,False,https://www.yelp.com/biz/bronco-burritos-galve...,+14097629344,(409) 762-9344,289.0,...,TX,"[4917 Broadway St, Galveston, TX 77551]",,29.291766,-94.822378,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,MvEBOqqAgMEd4AZ2KR8tww,the-wet-whistle-galveston,The Wet Whistle,https://s3-media3.fl.yelpcdn.com/bphoto/-Cl6AL...,True,False,https://www.yelp.com/biz/the-wet-whistle-galve...,+14095264433,(409) 526-4433,1.0,...,TX,"[21502 San Luis Pass, Galveston, TX 77554]",,29.141583,-95.040493,https://www.yelp.com/raq/MvEBOqqAgMEd4AZ2KR8tw...,Message the Business,NaN,NaN,NaN
167,TYsi4SYrkmLiGtPP5LE10g,bar-43-galveston,Bar 43,https://s3-media4.fl.yelpcdn.com/bphoto/8JNNli...,True,False,https://www.yelp.com/biz/bar-43-galveston?adju...,+14096322105,(409) 632-2105,1.0,...,TX,"[4302 Ave S, Galveston, TX 77550]",,29.280968,-94.810836,https://www.yelp.com/raq/TYsi4SYrkmLiGtPP5LE10...,Message the Business,NaN,NaN,NaN
168,SsmOfFeY0wdPB7ED26JTuQ,the-kings-court-freeport,The King's Court,https://s3-media3.fl.yelpcdn.com/bphoto/divNw7...,True,False,https://www.yelp.com/biz/the-kings-court-freep...,+17135182098,(713) 518-2098,3.0,...,TX,"[528 Gulf Blvd, Freeport, TX 77541]",,28.966447,-95.364985,NaN,NaN,NaN,NaN,NaN
169,ezMsWPV1G82ItSEZDFL_PA,lions-bar-b-q-katy,Lion's Bar-B-Q,https://s3-media4.fl.yelpcdn.com/bphoto/GlGFwt...,True,False,https://www.yelp.com/biz/lions-bar-b-q-katy?ad...,+18328200720,(832) 820-0720,1.0,...,TX,"[18309 Fm 529, Katy, TX 77449]",,29.875160,-95.770610,NaN,NaN,NaN,NaN,NaN


In [148]:
# Clean datasets. The "hours" column was messy and not in a string form, so I extracted information from the column and transited it to
# string form. Similar for the "transactions" column
galveston_open = []
galveston_hour_type = []
galveston_is_open_now = []
galveston_transactions = []
for i in range(galveston_details_final.shape[0]):
    if not isNaN(galveston_details_final["hours"].iloc[i]):
        galveston_open.append(galveston_details_final["hours"].iloc[i][0]["open"])
        galveston_hour_type.append(galveston_details_final["hours"].iloc[i][0]["hours_type"])
        galveston_is_open_now.append(galveston_details_final["hours"].iloc[i][0]["is_open_now"])
    else:
        galveston_open.append("")
        galveston_hour_type.append("")
        galveston_is_open_now.append("")
    if (galveston_details_final["transactions"].iloc[i] != []) and (not isNaN(galveston_details_final["transactions"].iloc[i])):
        galveston_transactions.append(galveston_details_final["transactions"].iloc[i][0])
    else:
        galveston_transactions.append("")

In [149]:
# I stored the info I extracted from "hours" and "transactions" form to new columns of the dataset
galveston_details_final["open"] = galveston_open
galveston_details_final["hour_type"] = galveston_hour_type
galveston_details_final["is_open_now"] = galveston_is_open_now
galveston_details_final["transactions"] = galveston_transactions
galveston_details_final["full_address"] = full_address_galveston
galveston_details_final.reset_index()

,index,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,...,coordinates.longitude,messaging.url,messaging.use_case_text,special_hours,error.code,error.description,open,hour_type,is_open_now,full_address
0,0,YAASmrPYP5Hp31kqadx__w,cajun-greek-galveston,Cajun Greek,https://s3-media3.fl.yelpcdn.com/bphoto/LKcJci...,True,False,https://www.yelp.com/biz/cajun-greek-galveston...,+14097447041,(409) 744-7041,...,-94.831641,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1600', 'end...",REGULAR,True,"2226 61st St Galveston, TX 77551"
1,1,CyhcrbaQ4Wjbud9rd9HJnQ,shrimp-n-stuff-galveston-4,Shrimp N Stuff,https://s3-media4.fl.yelpcdn.com/bphoto/oFhJ9E...,True,False,https://www.yelp.com/biz/shrimp-n-stuff-galves...,+14097632805,(409) 763-2805,...,-94.808440,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1030', 'end...",REGULAR,True,"3901 Ave O Galveston, TX 77550"
2,2,doSYFMDc3somcL4YKQb2FA,gypsy-joynt-galveston-2,Gypsy Joynt,https://s3-media1.fl.yelpcdn.com/bphoto/0fZzqv...,True,False,https://www.yelp.com/biz/gypsy-joynt-galveston...,+14094972069,(409) 497-2069,...,-94.831648,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1100', 'end...",REGULAR,True,"6105 Stewart Rd Galveston, TX 77551"
3,3,C4dQLeweVAez_gVuk9A60g,seahorse-grill-galveston-2,Seahorse Grill,https://s3-media2.fl.yelpcdn.com/bphoto/ZWXpfT...,True,False,https://www.yelp.com/biz/seahorse-grill-galves...,+14097418400,(409) 741-8400,...,-94.868038,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1100', 'end...",REGULAR,True,"3802 Cove View Blvd Ste C Galveston, TX 77554"
4,4,oxKepiR0j31RPOL9KcNwdQ,bronco-burritos-galveston,Bronco Burritos,https://s3-media2.fl.yelpcdn.com/bphoto/ouEDxP...,True,False,https://www.yelp.com/biz/bronco-burritos-galve...,+14097629344,(409) 762-9344,...,-94.822378,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '0600', 'end...",REGULAR,False,"4917 Broadway St Galveston, TX 77551"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,166,MvEBOqqAgMEd4AZ2KR8tww,the-wet-whistle-galveston,The Wet Whistle,https://s3-media3.fl.yelpcdn.com/bphoto/-Cl6AL...,True,False,https://www.yelp.com/biz/the-wet-whistle-galve...,+14095264433,(409) 526-4433,...,-95.040493,https://www.yelp.com/raq/MvEBOqqAgMEd4AZ2KR8tw...,Message the Business,NaN,NaN,NaN,"[{'is_overnight': True, 'start': '1600', 'end'...",REGULAR,True,"League City, TX 77573"
167,167,TYsi4SYrkmLiGtPP5LE10g,bar-43-galveston,Bar 43,https://s3-media4.fl.yelpcdn.com/bphoto/8JNNli...,True,False,https://www.yelp.com/biz/bar-43-galveston?adju...,+14096322105,(409) 632-2105,...,-94.810836,https://www.yelp.com/raq/TYsi4SYrkmLiGtPP5LE10...,Message the Business,NaN,NaN,NaN,"[{'is_overnight': True, 'start': '1200', 'end'...",REGULAR,True,"21502 San Luis Pass Galveston, TX 77554"
168,168,SsmOfFeY0wdPB7ED26JTuQ,the-kings-court-freeport,The King's Court,https://s3-media3.fl.yelpcdn.com/bphoto/divNw7...,True,False,https://www.yelp.com/biz/the-kings-court-freep...,+17135182098,(713) 518-2098,...,-95.364985,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1100', 'end...",REGULAR,True,"4302 Ave S Galveston, TX 77550"
169,169,ezMsWPV1G82ItSEZDFL_PA,lions-bar-b-q-katy,Lion's Bar-B-Q,https://s3-media4.fl.yelpcdn.com/bphoto/GlGFwt...,True,False,https://www.yelp.com/biz/lions-bar-b-q-katy?ad...,+18328200720,(832) 820-0720,...,-95.770610,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1800', 'end...",REGULAR,False,"528 Gulf Blvd Freeport, TX 77541"


In [153]:
# Transform the dataframe to a csv
galveston_details_final.to_csv("Galveston_Yelp_details.csv")

## Hilo's Business Details

In [26]:
# Use a list to store all json files of Hilo getting from Business details API
# for debug purpose, not using the data here
hilo_json_list = []
endpoint_url = "https://api.yelp.com/v3/businesses/"
for i in hilo_locations["id"]:
    endpoint_url_detail = endpoint_url + i
    res = requests.get(url=endpoint_url_detail,
                        headers={"Authorization":API_KEY}
                        )
    response = json.loads(res.content)
    hilo_json_list.append(response)

In [100]:
# Use a Pandas Dataframe to store all json files of Hilo getting from Business details API
hilo_df_list = []
endpoint_url = "https://api.yelp.com/v3/businesses/"
for i in hilo_locations["id"]:
    endpoint_url_detail = endpoint_url + i
    res = requests.get(url=endpoint_url_detail,
                        headers={"Authorization":API_KEY}
                        )
    df = json_normalize(res.json())
    hilo_df_list.append(df)
hilo_details = pd.concat(hilo_df_list)
hilo_details

,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,...,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude,messaging.url,messaging.use_case_text,special_hours,error.code,error.description
0,4kJ8m3FfW2SuiJ1gh44mfg,two-ladies-kitchen-hilo,Two Ladies Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/bhJ9wR...,True,False,https://www.yelp.com/biz/two-ladies-kitchen-hi...,+18089614766,(808) 961-4766,1098.0,...,HI,"[274 Kilauea Ave, Hilo, HI 96720]",,19.721042,-155.084300,NaN,NaN,NaN,NaN,NaN
0,OCtBAGJ1OJT2NTTvwEw84A,hawaiian-style-cafe-hilo-hilo,Hawaiian Style Cafe Hilo,https://s3-media4.fl.yelpcdn.com/bphoto/lNwVH0...,True,False,https://www.yelp.com/biz/hawaiian-style-cafe-h...,+18089699265,(808) 969-9265,1804.0,...,HI,"[681 Manono St, Ste 101, Hilo, HI 96720]",,19.713019,-155.069893,NaN,NaN,NaN,NaN,NaN
0,WP2ZjFW41wUK0qeilF--dA,pineapples-island-fresh-cuisine-hilo,Pineapples Island Fresh Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/5T3egR...,True,False,https://www.yelp.com/biz/pineapples-island-fre...,+18082385324,(808) 238-5324,1854.0,...,HI,"[332 Keawe St, Hilo, HI 96720]",,19.722740,-155.086160,NaN,NaN,NaN,NaN,NaN
0,9qV9988-I90odzitG-G61w,cafe-pesto-hilo-bay-hilo-3,Cafe Pesto Hilo Bay,https://s3-media2.fl.yelpcdn.com/bphoto/cw1f_x...,True,False,https://www.yelp.com/biz/cafe-pesto-hilo-bay-h...,+18089696640,(808) 969-6640,1440.0,...,HI,"[308 Kamehameha Ave, Ste 101, Hilo, HI 96720]",,19.723753,-155.085017,NaN,NaN,NaN,NaN,NaN
0,m7P7_44_awlR6XvAg31ctg,pauls-place-hilo,Paul's Place,https://s3-media4.fl.yelpcdn.com/bphoto/mwzdwT...,True,False,https://www.yelp.com/biz/pauls-place-hilo?adju...,+18082808646,(808) 280-8646,446.0,...,HI,"[132 Punahoa St, Hilo, HI 96720]",,19.722601,-155.084259,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,48HbDoyK37JS39xulFbuxQ,pizza-hut-pahoa,Pizza Hut,https://s3-media2.fl.yelpcdn.com/bphoto/Z6jOE_...,True,False,https://www.yelp.com/biz/pizza-hut-pahoa?adjus...,+18086431111,(808) 643-1111,10.0,...,HI,"[15-2714 Pahoa Village Rd, Bldg D Unit 4, Paho...",,19.502885,-154.956514,NaN,NaN,NaN,NaN,NaN
0,OwJMxmj6CpWgV18jq2pIAg,mcdonalds-pāhoa,McDonald's,https://s3-media2.fl.yelpcdn.com/bphoto/mroOYG...,True,False,https://www.yelp.com/biz/mcdonalds-p%C4%81hoa?...,,,5.0,...,HI,"[15-2714 Pahoa Village Rd, Pāhoa, HI 96778]",,19.502874,-154.955653,NaN,NaN,NaN,NaN,NaN
0,YO14nJRmwfFjG2ieszseSw,plates-of-aloha-waimea,Plates Of Aloha,https://s3-media2.fl.yelpcdn.com/bphoto/r2R0Id...,True,False,https://www.yelp.com/biz/plates-of-aloha-waime...,+18089382948,(808) 938-2948,12.0,...,HI,"[65-1214 Lindsay Rd, Waimea, HI 96743]",,20.021581,-155.669409,NaN,NaN,NaN,NaN,NaN
0,_BMxln7D-jwBpwSY7_rf5A,big-island-tea-mountain-view-3,Big Island Tea,,False,False,https://www.yelp.com/biz/big-island-tea-mounta...,+18089361172,(808) 936-1172,1.0,...,HI,"[18-2465 N Glenwood Rd, Mountain View, HI 96771]",,19.504640,-155.177420,NaN,NaN,NaN,NaN,NaN


In [101]:
# Get all rows that did not get accurate information due to "too many requests" error
# I need to replace these rows with error messages to rows with accurate business details information
hilo_nan = []
for i in range(len(list(hilo_details["name"]))):
    if isNaN(list(hilo_details["name"])[i]):
        hilo_nan.append(i)
hilo_nan

[197, 280, 319, 321]

In [102]:
# Get the accurate information of the rows with the error messages
# based on the index I got from the previous function
hilo_nan_detail = []
endpoint_url = "https://api.yelp.com/v3/businesses/"
for i in hilo_nan:
    id_num = hilo_locations.iloc[i]["id"]
    endpoint_url_detail = endpoint_url + id_num
    res = requests.get(url=endpoint_url_detail,
                        headers={"Authorization":API_KEY}
                        )
    response = json_normalize(res.json())
    hilo_nan_detail.append(response)
hilo_nan_detail

[                       id                        alias  \
 0  UOGzckX_5HvLfOYKjcrvQw  big-island-birria-boss-hilo   
 
                      name                                          image_url  \
 0  Big Island Birria Boss  https://s3-media3.fl.yelpcdn.com/bphoto/F8i_A_...   
 
    is_claimed  is_closed                                                url  \
 0        True      False  https://www.yelp.com/biz/big-island-birria-bos...   
 
           phone   display_phone  review_count  ... location.address2  \
 0  +12094997500  (209) 499-7500             9  ...                     
 
    location.address3 location.city location.zip_code location.country  \
 0               None          Hilo             96720               US   
 
   location.state             location.display_address location.cross_streets  \
 0             HI  [380 Kanoelehua Rd, Hilo, HI 96720]                          
 
   coordinates.latitude coordinates.longitude  
 0             19.71732            -155.0650

In [111]:
# Concat the rows I got from the previous cell
hilo_nan_detail_df = pd.concat(hilo_nan_detail)
hilo_nan_detail_df

,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,...,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude,price
0,UOGzckX_5HvLfOYKjcrvQw,big-island-birria-boss-hilo,Big Island Birria Boss,https://s3-media3.fl.yelpcdn.com/bphoto/F8i_A_...,True,False,https://www.yelp.com/biz/big-island-birria-bos...,+12094997500,(209) 499-7500,9,...,None,Hilo,96720,US,HI,"[380 Kanoelehua Rd, Hilo, HI 96720]",,19.717320,-155.065020,NaN
0,_2rBthT1z5GSxhPpTo-ICw,l-and-l-hawaiian-barbecue-keaau,L&L Hawaiian Barbecue,https://s3-media4.fl.yelpcdn.com/bphoto/l9vaXw...,True,False,https://www.yelp.com/biz/l-and-l-hawaiian-barb...,+18089826668,(808) 982-6668,33,...,,Keaau,96749,US,HI,"[16-586 Old Volcano Rd, Ste 104, Keaau, HI 96749]",,19.622269,-155.040777,$
0,zbKKMV0TWCQSCvg1bwkDng,la-hiki-ola-kava-bar-pāhoa,La Hiki Ola Kava Bar,https://s3-media2.fl.yelpcdn.com/bphoto/VpTIeo...,True,False,https://www.yelp.com/biz/la-hiki-ola-kava-bar-...,+18089650615,(808) 965-0615,12,...,,Pāhoa,96778,US,HI,"[15-2950 Pahoa Village Rd, Pāhoa, HI 96778]",,19.493789,-154.945199,$
0,xT9fpNptk2_-elsM4um_jQ,ahualani-park-hilo,Ahualani Park,,False,False,https://www.yelp.com/biz/ahualani-park-hilo?ad...,,,2,...,,Hilo,96720,US,HI,"[1516 Kikaha St, Hilo, HI 96720]",,19.677794,-155.096787,NaN


In [108]:
# Drop the lines with error messages
hilo_details = hilo_details.reset_index()
hilo_details = hilo_details.drop(hilo_nan)
hilo_details

,level_0,index,id,alias,name,image_url,is_claimed,is_closed,url,phone,...,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude,messaging.url,messaging.use_case_text,special_hours,error.code,error.description
0,0,0,4kJ8m3FfW2SuiJ1gh44mfg,two-ladies-kitchen-hilo,Two Ladies Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/bhJ9wR...,True,False,https://www.yelp.com/biz/two-ladies-kitchen-hi...,+18089614766,...,HI,"[274 Kilauea Ave, Hilo, HI 96720]",,19.721042,-155.084300,NaN,NaN,NaN,NaN,NaN
1,1,0,OCtBAGJ1OJT2NTTvwEw84A,hawaiian-style-cafe-hilo-hilo,Hawaiian Style Cafe Hilo,https://s3-media4.fl.yelpcdn.com/bphoto/lNwVH0...,True,False,https://www.yelp.com/biz/hawaiian-style-cafe-h...,+18089699265,...,HI,"[681 Manono St, Ste 101, Hilo, HI 96720]",,19.713019,-155.069893,NaN,NaN,NaN,NaN,NaN
2,2,0,WP2ZjFW41wUK0qeilF--dA,pineapples-island-fresh-cuisine-hilo,Pineapples Island Fresh Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/5T3egR...,True,False,https://www.yelp.com/biz/pineapples-island-fre...,+18082385324,...,HI,"[332 Keawe St, Hilo, HI 96720]",,19.722740,-155.086160,NaN,NaN,NaN,NaN,NaN
3,3,0,9qV9988-I90odzitG-G61w,cafe-pesto-hilo-bay-hilo-3,Cafe Pesto Hilo Bay,https://s3-media2.fl.yelpcdn.com/bphoto/cw1f_x...,True,False,https://www.yelp.com/biz/cafe-pesto-hilo-bay-h...,+18089696640,...,HI,"[308 Kamehameha Ave, Ste 101, Hilo, HI 96720]",,19.723753,-155.085017,NaN,NaN,NaN,NaN,NaN
4,4,0,m7P7_44_awlR6XvAg31ctg,pauls-place-hilo,Paul's Place,https://s3-media4.fl.yelpcdn.com/bphoto/mwzdwT...,True,False,https://www.yelp.com/biz/pauls-place-hilo?adju...,+18082808646,...,HI,"[132 Punahoa St, Hilo, HI 96720]",,19.722601,-155.084259,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,373,0,48HbDoyK37JS39xulFbuxQ,pizza-hut-pahoa,Pizza Hut,https://s3-media2.fl.yelpcdn.com/bphoto/Z6jOE_...,True,False,https://www.yelp.com/biz/pizza-hut-pahoa?adjus...,+18086431111,...,HI,"[15-2714 Pahoa Village Rd, Bldg D Unit 4, Paho...",,19.502885,-154.956514,NaN,NaN,NaN,NaN,NaN
374,374,0,OwJMxmj6CpWgV18jq2pIAg,mcdonalds-pāhoa,McDonald's,https://s3-media2.fl.yelpcdn.com/bphoto/mroOYG...,True,False,https://www.yelp.com/biz/mcdonalds-p%C4%81hoa?...,,...,HI,"[15-2714 Pahoa Village Rd, Pāhoa, HI 96778]",,19.502874,-154.955653,NaN,NaN,NaN,NaN,NaN
375,375,0,YO14nJRmwfFjG2ieszseSw,plates-of-aloha-waimea,Plates Of Aloha,https://s3-media2.fl.yelpcdn.com/bphoto/r2R0Id...,True,False,https://www.yelp.com/biz/plates-of-aloha-waime...,+18089382948,...,HI,"[65-1214 Lindsay Rd, Waimea, HI 96743]",,20.021581,-155.669409,NaN,NaN,NaN,NaN,NaN
376,376,0,_BMxln7D-jwBpwSY7_rf5A,big-island-tea-mountain-view-3,Big Island Tea,,False,False,https://www.yelp.com/biz/big-island-tea-mounta...,+18089361172,...,HI,"[18-2465 N Glenwood Rd, Mountain View, HI 96771]",,19.504640,-155.177420,NaN,NaN,NaN,NaN,NaN


In [76]:
# Concat the rows I got from the previous cell and reset index
hilo_nan_detail_df = pd.concat(hilo_nan_detail)
# The index here is based on the indexes I detected before
# Each time the indexes may change, so it may be needed to manually change
# the index in the list here
hilo_nan_detail_df["index"] = [52, 156, 288, 318, 319, 340]
hilo_nan_detail_df.set_index("index")

,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,...,location.city,location.zip_code,location.country,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude,messaging.url,messaging.use_case_text
index,,,,,,,,,,,,,,,,,,,,,
52,nGh_8FzSq8PHtFFXQ8ydSA,le-yellow-sub-hilo,Le Yellow Sub,https://s3-media3.fl.yelpcdn.com/bphoto/kSgBT8...,True,False,https://www.yelp.com/biz/le-yellow-sub-hilo?ad...,+18089370911,(808) 937-0911,71,...,Hilo,96720,US,HI,"[213 Kalalaniana Ave, Hilo, HI 96720]",,19.717840,-155.087936,NaN,NaN
156,WHxNRdWdkBxwBT5OBnlV3Q,seoul-station-hilo,Seoul Station,https://s3-media3.fl.yelpcdn.com/bphoto/b-frZZ...,True,False,https://www.yelp.com/biz/seoul-station-hilo?ad...,+18089339301,(808) 933-9301,37,...,Hilo,96720,US,HI,"[417 Kalanikoa St, Hilo, HI 96720]",,19.716950,-155.065511,https://www.yelp.com/raq/WHxNRdWdkBxwBT5OBnlV3...,Message the Business
288,2z7pqa9HsLJhkbzYMDQmwA,texas-island-bbq-keaau-2,Texas Island BBQ,https://s3-media1.fl.yelpcdn.com/bphoto/zuKDPw...,True,False,https://www.yelp.com/biz/texas-island-bbq-keaa...,+18083199131,(808) 319-9131,30,...,Keaau,96749,US,HI,"[15-1594 4th Ave, Keaau, HI 96749]",,19.605462,-154.960953,NaN,NaN
318,DRw2jXygsbaouNWLJ51kkQ,comechee-iceshave-hilo,CoMechee Iceshave,https://s3-media2.fl.yelpcdn.com/bphoto/bMhQYr...,True,False,https://www.yelp.com/biz/comechee-iceshave-hil...,+18087857516,(808) 785-7516,1,...,Hilo,96720,US,HI,"[213 Kalanianaole St, Hilo, HI 96720]",,19.724548,-155.060890,https://www.yelp.com/raq/DRw2jXygsbaouNWLJ51kk...,Message the Business
319,zbKKMV0TWCQSCvg1bwkDng,la-hiki-ola-kava-bar-pāhoa,La Hiki Ola Kava Bar,https://s3-media2.fl.yelpcdn.com/bphoto/VpTIeo...,True,False,https://www.yelp.com/biz/la-hiki-ola-kava-bar-...,+18089650615,(808) 965-0615,12,...,Pāhoa,96778,US,HI,"[15-2950 Pahoa Village Rd, Pāhoa, HI 96778]",,19.493789,-154.945199,NaN,NaN
340,iEzyUF22gyoVWjP0gYzWgQ,kealoha-beach-park-hilo,Kealoha Beach Park,https://s3-media1.fl.yelpcdn.com/bphoto/pMQR1s...,False,False,https://www.yelp.com/biz/kealoha-beach-park-hi...,,,1,...,Hilo,96720,US,HI,"[1701 Kalanianaole Ave, Hilo, HI 96720]",,19.735152,-155.033602,NaN,NaN


In [120]:
# Drop the columns without real information
hilo_details_new = hilo_details.iloc[:,2:]
hilo_details_new

,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,...,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude,messaging.url,messaging.use_case_text,special_hours,error.code,error.description
0,4kJ8m3FfW2SuiJ1gh44mfg,two-ladies-kitchen-hilo,Two Ladies Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/bhJ9wR...,True,False,https://www.yelp.com/biz/two-ladies-kitchen-hi...,+18089614766,(808) 961-4766,1098.0,...,HI,"[274 Kilauea Ave, Hilo, HI 96720]",,19.721042,-155.084300,NaN,NaN,NaN,NaN,NaN
1,OCtBAGJ1OJT2NTTvwEw84A,hawaiian-style-cafe-hilo-hilo,Hawaiian Style Cafe Hilo,https://s3-media4.fl.yelpcdn.com/bphoto/lNwVH0...,True,False,https://www.yelp.com/biz/hawaiian-style-cafe-h...,+18089699265,(808) 969-9265,1804.0,...,HI,"[681 Manono St, Ste 101, Hilo, HI 96720]",,19.713019,-155.069893,NaN,NaN,NaN,NaN,NaN
2,WP2ZjFW41wUK0qeilF--dA,pineapples-island-fresh-cuisine-hilo,Pineapples Island Fresh Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/5T3egR...,True,False,https://www.yelp.com/biz/pineapples-island-fre...,+18082385324,(808) 238-5324,1854.0,...,HI,"[332 Keawe St, Hilo, HI 96720]",,19.722740,-155.086160,NaN,NaN,NaN,NaN,NaN
3,9qV9988-I90odzitG-G61w,cafe-pesto-hilo-bay-hilo-3,Cafe Pesto Hilo Bay,https://s3-media2.fl.yelpcdn.com/bphoto/cw1f_x...,True,False,https://www.yelp.com/biz/cafe-pesto-hilo-bay-h...,+18089696640,(808) 969-6640,1440.0,...,HI,"[308 Kamehameha Ave, Ste 101, Hilo, HI 96720]",,19.723753,-155.085017,NaN,NaN,NaN,NaN,NaN
4,m7P7_44_awlR6XvAg31ctg,pauls-place-hilo,Paul's Place,https://s3-media4.fl.yelpcdn.com/bphoto/mwzdwT...,True,False,https://www.yelp.com/biz/pauls-place-hilo?adju...,+18082808646,(808) 280-8646,446.0,...,HI,"[132 Punahoa St, Hilo, HI 96720]",,19.722601,-155.084259,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,48HbDoyK37JS39xulFbuxQ,pizza-hut-pahoa,Pizza Hut,https://s3-media2.fl.yelpcdn.com/bphoto/Z6jOE_...,True,False,https://www.yelp.com/biz/pizza-hut-pahoa?adjus...,+18086431111,(808) 643-1111,10.0,...,HI,"[15-2714 Pahoa Village Rd, Bldg D Unit 4, Paho...",,19.502885,-154.956514,NaN,NaN,NaN,NaN,NaN
374,OwJMxmj6CpWgV18jq2pIAg,mcdonalds-pāhoa,McDonald's,https://s3-media2.fl.yelpcdn.com/bphoto/mroOYG...,True,False,https://www.yelp.com/biz/mcdonalds-p%C4%81hoa?...,,,5.0,...,HI,"[15-2714 Pahoa Village Rd, Pāhoa, HI 96778]",,19.502874,-154.955653,NaN,NaN,NaN,NaN,NaN
375,YO14nJRmwfFjG2ieszseSw,plates-of-aloha-waimea,Plates Of Aloha,https://s3-media2.fl.yelpcdn.com/bphoto/r2R0Id...,True,False,https://www.yelp.com/biz/plates-of-aloha-waime...,+18089382948,(808) 938-2948,12.0,...,HI,"[65-1214 Lindsay Rd, Waimea, HI 96743]",,20.021581,-155.669409,NaN,NaN,NaN,NaN,NaN
376,_BMxln7D-jwBpwSY7_rf5A,big-island-tea-mountain-view-3,Big Island Tea,,False,False,https://www.yelp.com/biz/big-island-tea-mounta...,+18089361172,(808) 936-1172,1.0,...,HI,"[18-2465 N Glenwood Rd, Mountain View, HI 96771]",,19.504640,-155.177420,NaN,NaN,NaN,NaN,NaN


In [124]:
# Merge the rows with the accurate info I missed before to the original dataset
hilo_details_final = pd.concat([hilo_details_new, hilo_nan_detail_df])
hilo_details_final = hilo_details_final.reset_index()

In [126]:
# Drop the columns without real information
hilo_details_final = hilo_details_final.iloc[:,1:]
hilo_details_final

,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,...,location.state,location.display_address,location.cross_streets,coordinates.latitude,coordinates.longitude,messaging.url,messaging.use_case_text,special_hours,error.code,error.description
0,4kJ8m3FfW2SuiJ1gh44mfg,two-ladies-kitchen-hilo,Two Ladies Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/bhJ9wR...,True,False,https://www.yelp.com/biz/two-ladies-kitchen-hi...,+18089614766,(808) 961-4766,1098.0,...,HI,"[274 Kilauea Ave, Hilo, HI 96720]",,19.721042,-155.084300,NaN,NaN,NaN,NaN,NaN
1,OCtBAGJ1OJT2NTTvwEw84A,hawaiian-style-cafe-hilo-hilo,Hawaiian Style Cafe Hilo,https://s3-media4.fl.yelpcdn.com/bphoto/lNwVH0...,True,False,https://www.yelp.com/biz/hawaiian-style-cafe-h...,+18089699265,(808) 969-9265,1804.0,...,HI,"[681 Manono St, Ste 101, Hilo, HI 96720]",,19.713019,-155.069893,NaN,NaN,NaN,NaN,NaN
2,WP2ZjFW41wUK0qeilF--dA,pineapples-island-fresh-cuisine-hilo,Pineapples Island Fresh Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/5T3egR...,True,False,https://www.yelp.com/biz/pineapples-island-fre...,+18082385324,(808) 238-5324,1854.0,...,HI,"[332 Keawe St, Hilo, HI 96720]",,19.722740,-155.086160,NaN,NaN,NaN,NaN,NaN
3,9qV9988-I90odzitG-G61w,cafe-pesto-hilo-bay-hilo-3,Cafe Pesto Hilo Bay,https://s3-media2.fl.yelpcdn.com/bphoto/cw1f_x...,True,False,https://www.yelp.com/biz/cafe-pesto-hilo-bay-h...,+18089696640,(808) 969-6640,1440.0,...,HI,"[308 Kamehameha Ave, Ste 101, Hilo, HI 96720]",,19.723753,-155.085017,NaN,NaN,NaN,NaN,NaN
4,m7P7_44_awlR6XvAg31ctg,pauls-place-hilo,Paul's Place,https://s3-media4.fl.yelpcdn.com/bphoto/mwzdwT...,True,False,https://www.yelp.com/biz/pauls-place-hilo?adju...,+18082808646,(808) 280-8646,446.0,...,HI,"[132 Punahoa St, Hilo, HI 96720]",,19.722601,-155.084259,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,2RScELF68G3n8B3lY0blpA,under-the-bodhi-tree-pāhoa,Under The Bodhi Tree,https://s3-media3.fl.yelpcdn.com/bphoto/YX3WX2...,True,False,https://www.yelp.com/biz/under-the-bodhi-tree-...,,,1.0,...,HI,"[15-2714 Pahoa Village Rd, D1, Pāhoa, HI 96778]",,19.502857,-154.955671,https://www.yelp.com/raq/2RScELF68G3n8B3lY0blp...,Message the Business,NaN,NaN,NaN
374,UOGzckX_5HvLfOYKjcrvQw,big-island-birria-boss-hilo,Big Island Birria Boss,https://s3-media3.fl.yelpcdn.com/bphoto/F8i_A_...,True,False,https://www.yelp.com/biz/big-island-birria-bos...,+12094997500,(209) 499-7500,9.0,...,HI,"[380 Kanoelehua Rd, Hilo, HI 96720]",,19.717320,-155.065020,NaN,NaN,NaN,NaN,NaN
375,_2rBthT1z5GSxhPpTo-ICw,l-and-l-hawaiian-barbecue-keaau,L&L Hawaiian Barbecue,https://s3-media4.fl.yelpcdn.com/bphoto/l9vaXw...,True,False,https://www.yelp.com/biz/l-and-l-hawaiian-barb...,+18089826668,(808) 982-6668,33.0,...,HI,"[16-586 Old Volcano Rd, Ste 104, Keaau, HI 96749]",,19.622269,-155.040777,NaN,NaN,NaN,NaN,NaN
376,zbKKMV0TWCQSCvg1bwkDng,la-hiki-ola-kava-bar-pāhoa,La Hiki Ola Kava Bar,https://s3-media2.fl.yelpcdn.com/bphoto/VpTIeo...,True,False,https://www.yelp.com/biz/la-hiki-ola-kava-bar-...,+18089650615,(808) 965-0615,12.0,...,HI,"[15-2950 Pahoa Village Rd, Pāhoa, HI 96778]",,19.493789,-154.945199,NaN,NaN,NaN,NaN,NaN


In [127]:
# Clean datasets. The "hours" column was messy and not in a string form, so I extracted information from the column and transited it to
# string form. Similar for the "transactions" column
hilo_open = []
hilo_hour_type = []
hilo_is_open_now = []
hilo_transactions = []
for i in range(hilo_details_final.shape[0]):
    if not isNaN(hilo_details_final["hours"].iloc[i]):
        hilo_open.append(hilo_details_final["hours"].iloc[i][0]["open"])
        hilo_hour_type.append(hilo_details_final["hours"].iloc[i][0]["hours_type"])
        hilo_is_open_now.append(hilo_details_final["hours"].iloc[i][0]["is_open_now"])
    else:
        hilo_open.append("")
        hilo_hour_type.append("")
        hilo_is_open_now.append("")
    if (hilo_details_final["transactions"].iloc[i] != []) and (not isNaN(hilo_details_final["transactions"].iloc[i])):
        hilo_transactions.append(hilo_details_final["transactions"].iloc[i][0])
    else:
        hilo_transactions.append("")

In [128]:
# I stored the info I extracted from "hours" and "transactions" form to new columns of the dataset
hilo_details_final["open"] = hilo_open
hilo_details_final["hour_type"] = hilo_hour_type
hilo_details_final["is_open_now"] = hilo_is_open_now
hilo_details_final["transactions"] = hilo_transactions
hilo_details_final["full_address"] = full_address_hilo
hilo_details_final = hilo_details_final.reset_index()
hilo_details_final

,index,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,...,coordinates.longitude,messaging.url,messaging.use_case_text,special_hours,error.code,error.description,open,hour_type,is_open_now,full_address
0,0,4kJ8m3FfW2SuiJ1gh44mfg,two-ladies-kitchen-hilo,Two Ladies Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/bhJ9wR...,True,False,https://www.yelp.com/biz/two-ladies-kitchen-hi...,+18089614766,(808) 961-4766,...,-155.084300,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1000', 'end...",REGULAR,False,"274 Kilauea Ave Hilo, HI 96720"
1,1,OCtBAGJ1OJT2NTTvwEw84A,hawaiian-style-cafe-hilo-hilo,Hawaiian Style Cafe Hilo,https://s3-media4.fl.yelpcdn.com/bphoto/lNwVH0...,True,False,https://www.yelp.com/biz/hawaiian-style-cafe-h...,+18089699265,(808) 969-9265,...,-155.069893,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '0700', 'end...",REGULAR,True,"681 Manono St Ste 101 Hilo, HI 96720"
2,2,WP2ZjFW41wUK0qeilF--dA,pineapples-island-fresh-cuisine-hilo,Pineapples Island Fresh Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/5T3egR...,True,False,https://www.yelp.com/biz/pineapples-island-fre...,+18082385324,(808) 238-5324,...,-155.086160,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1100', 'end...",REGULAR,True,"332 Keawe St Hilo, HI 96720"
3,3,9qV9988-I90odzitG-G61w,cafe-pesto-hilo-bay-hilo-3,Cafe Pesto Hilo Bay,https://s3-media2.fl.yelpcdn.com/bphoto/cw1f_x...,True,False,https://www.yelp.com/biz/cafe-pesto-hilo-bay-h...,+18089696640,(808) 969-6640,...,-155.085017,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1100', 'end...",REGULAR,True,"308 Kamehameha Ave Ste 101 Hilo, HI 96720"
4,4,m7P7_44_awlR6XvAg31ctg,pauls-place-hilo,Paul's Place,https://s3-media4.fl.yelpcdn.com/bphoto/mwzdwT...,True,False,https://www.yelp.com/biz/pauls-place-hilo?adju...,+18082808646,(808) 280-8646,...,-155.084259,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '0730', 'end...",REGULAR,False,"132 Punahoa St Hilo, HI 96720"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,373,2RScELF68G3n8B3lY0blpA,under-the-bodhi-tree-pāhoa,Under The Bodhi Tree,https://s3-media3.fl.yelpcdn.com/bphoto/YX3WX2...,True,False,https://www.yelp.com/biz/under-the-bodhi-tree-...,,,...,-154.955671,https://www.yelp.com/raq/2RScELF68G3n8B3lY0blp...,Message the Business,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1100', 'end...",REGULAR,True,"15-2714 Pahoa Village Rd Bldg D Unit 4 Pahoa, ..."
374,374,UOGzckX_5HvLfOYKjcrvQw,big-island-birria-boss-hilo,Big Island Birria Boss,https://s3-media3.fl.yelpcdn.com/bphoto/F8i_A_...,True,False,https://www.yelp.com/biz/big-island-birria-bos...,+12094997500,(209) 499-7500,...,-155.065020,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1000', 'end...",REGULAR,False,"15-2714 Pahoa Village Rd Pāhoa, HI 96778"
375,375,_2rBthT1z5GSxhPpTo-ICw,l-and-l-hawaiian-barbecue-keaau,L&L Hawaiian Barbecue,https://s3-media4.fl.yelpcdn.com/bphoto/l9vaXw...,True,False,https://www.yelp.com/biz/l-and-l-hawaiian-barb...,+18089826668,(808) 982-6668,...,-155.040777,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '0900', 'end...",REGULAR,True,"65-1214 Lindsay Rd Waimea, HI 96743"
376,376,zbKKMV0TWCQSCvg1bwkDng,la-hiki-ola-kava-bar-pāhoa,La Hiki Ola Kava Bar,https://s3-media2.fl.yelpcdn.com/bphoto/VpTIeo...,True,False,https://www.yelp.com/biz/la-hiki-ola-kava-bar-...,+18089650615,(808) 965-0615,...,-154.945199,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '0930', 'end...",REGULAR,True,"18-2465 N Glenwood Rd Mountain View, HI 96771"


In [150]:
# Drop the columns without real information
hilo_details_final = hilo_details_final.iloc[:,1:]
hilo_details_final

,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,...,coordinates.longitude,messaging.url,messaging.use_case_text,special_hours,error.code,error.description,open,hour_type,is_open_now,full_address
0,4kJ8m3FfW2SuiJ1gh44mfg,two-ladies-kitchen-hilo,Two Ladies Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/bhJ9wR...,True,False,https://www.yelp.com/biz/two-ladies-kitchen-hi...,+18089614766,(808) 961-4766,1098.0,...,-155.084300,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1000', 'end...",REGULAR,False,"274 Kilauea Ave Hilo, HI 96720"
1,OCtBAGJ1OJT2NTTvwEw84A,hawaiian-style-cafe-hilo-hilo,Hawaiian Style Cafe Hilo,https://s3-media4.fl.yelpcdn.com/bphoto/lNwVH0...,True,False,https://www.yelp.com/biz/hawaiian-style-cafe-h...,+18089699265,(808) 969-9265,1804.0,...,-155.069893,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '0700', 'end...",REGULAR,True,"681 Manono St Ste 101 Hilo, HI 96720"
2,WP2ZjFW41wUK0qeilF--dA,pineapples-island-fresh-cuisine-hilo,Pineapples Island Fresh Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/5T3egR...,True,False,https://www.yelp.com/biz/pineapples-island-fre...,+18082385324,(808) 238-5324,1854.0,...,-155.086160,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1100', 'end...",REGULAR,True,"332 Keawe St Hilo, HI 96720"
3,9qV9988-I90odzitG-G61w,cafe-pesto-hilo-bay-hilo-3,Cafe Pesto Hilo Bay,https://s3-media2.fl.yelpcdn.com/bphoto/cw1f_x...,True,False,https://www.yelp.com/biz/cafe-pesto-hilo-bay-h...,+18089696640,(808) 969-6640,1440.0,...,-155.085017,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1100', 'end...",REGULAR,True,"308 Kamehameha Ave Ste 101 Hilo, HI 96720"
4,m7P7_44_awlR6XvAg31ctg,pauls-place-hilo,Paul's Place,https://s3-media4.fl.yelpcdn.com/bphoto/mwzdwT...,True,False,https://www.yelp.com/biz/pauls-place-hilo?adju...,+18082808646,(808) 280-8646,446.0,...,-155.084259,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '0730', 'end...",REGULAR,False,"132 Punahoa St Hilo, HI 96720"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,2RScELF68G3n8B3lY0blpA,under-the-bodhi-tree-pāhoa,Under The Bodhi Tree,https://s3-media3.fl.yelpcdn.com/bphoto/YX3WX2...,True,False,https://www.yelp.com/biz/under-the-bodhi-tree-...,,,1.0,...,-154.955671,https://www.yelp.com/raq/2RScELF68G3n8B3lY0blp...,Message the Business,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1100', 'end...",REGULAR,True,"15-2714 Pahoa Village Rd Bldg D Unit 4 Pahoa, ..."
374,UOGzckX_5HvLfOYKjcrvQw,big-island-birria-boss-hilo,Big Island Birria Boss,https://s3-media3.fl.yelpcdn.com/bphoto/F8i_A_...,True,False,https://www.yelp.com/biz/big-island-birria-bos...,+12094997500,(209) 499-7500,9.0,...,-155.065020,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '1000', 'end...",REGULAR,False,"15-2714 Pahoa Village Rd Pāhoa, HI 96778"
375,_2rBthT1z5GSxhPpTo-ICw,l-and-l-hawaiian-barbecue-keaau,L&L Hawaiian Barbecue,https://s3-media4.fl.yelpcdn.com/bphoto/l9vaXw...,True,False,https://www.yelp.com/biz/l-and-l-hawaiian-barb...,+18089826668,(808) 982-6668,33.0,...,-155.040777,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '0900', 'end...",REGULAR,True,"65-1214 Lindsay Rd Waimea, HI 96743"
376,zbKKMV0TWCQSCvg1bwkDng,la-hiki-ola-kava-bar-pāhoa,La Hiki Ola Kava Bar,https://s3-media2.fl.yelpcdn.com/bphoto/VpTIeo...,True,False,https://www.yelp.com/biz/la-hiki-ola-kava-bar-...,+18089650615,(808) 965-0615,12.0,...,-154.945199,NaN,NaN,NaN,NaN,NaN,"[{'is_overnight': False, 'start': '0930', 'end...",REGULAR,True,"18-2465 N Glenwood Rd Mountain View, HI 96771"


In [154]:
# Transform the dataframe to a csv
hilo_details_final.to_csv("Hilo_Yelp_details.csv")